In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime


In [ ]:
def get_penalty_amount(bg):
  if ((bg > 200) & (bg < 5001)):
    return( bg*0.02 )

  if ((bg > 5000) & (bg < 10001)):
    return(bg*0.04)

  if (bg > 10000):
    return(bg*0.06)
  else:
    return(0)

def get_yep(bg, sub, yep_check):
  yep_princ_amount = 0
  if (bg > 10000):
    yep_princ_amount = bg*0.06
  else:
    yep_princ_amount = 0

  yep_sub_amount = 0
  if (sub > 10000):
    yep_sub_amount = sub*0.06
  else:
    yep_sub_amount = 0

  if (str(yep_check).lower() == 'true'):
    return (yep_princ_amount + yep_sub_amount)
  else:
    return 0
  

def get_datepart(type, date):
  dt = datetime.strptime(date, '%Y-%m-%d')
  if (type.lower() == 'day'):
    return dt.day
  if (type.lower() == 'month'):
    return dt.month
  if (type.lower() == 'year'):
    return dt.year

def get_total_penalty(bg, subs, yep):
  penalty_amount = get_penalty_amount(bg)
  year_end_pen = get_yep(bg, subs, yep)
  total_penalty = penalty_amount + year_end_pen
  return (total_penalty)

def get_total_interest(subs, interest, start_date, end_date):
  per_diem_year = 360
  per_diem_month = 30

  days_interest_year = per_diem_year * (get_datepart('year', start_date) - get_datepart('year', end_date))
  days_interest_month = per_diem_month * (get_datepart('month', start_date) - get_datepart('month', end_date))
  days_interest_day = (get_datepart('day', start_date) - get_datepart('day', end_date))
  total_days = days_interest_day + days_interest_month + days_interest_year

  sub_per_diem = total_days / per_diem_year
  total_interest = abs(sub_per_diem * subs * interest)
  return (total_interest)
  

In [ ]:
test = get_total_penalty(70000, 500, 'false')
print (test)

test = get_total_interest(123.72, 0.18, '2022-07-20', '2023-02-11')
print(test)

4200.0
12.43386


In [ ]:
df = pd.read_excel('test.xlsx')


In [ ]:
test = df[['PARCEL', 'FEES: CATEGORY.1', 'FEES: AMOUNT.1','FEES: INTEREST', 'FEES: EFFECTIVE DATE', 'FEES: END DATE', 'FEES: INTEREST.1', 'BEGINNING BALANCE']]
test = test[test['FEES: CATEGORY.1'] == 'Subsequent Tax'].dropna()

test['FEES: EFFECTIVE DATE'] = pd.to_datetime(test['FEES: EFFECTIVE DATE'], format='%m/%d/%Y')
test['FEES: END DATE'] = pd.to_datetime(test['FEES: END DATE'], format='%m/%d/%Y')
test['FEES: EFFECTIVE DATE'] = test['FEES: EFFECTIVE DATE'].dt.strftime('%Y-%m-%d')
test['FEES: END DATE'] = test['FEES: END DATE'].dt.strftime('%Y-%m-%d')

In [ ]:
from pandas.core.reshape.melt import tile_compat
total_penalty = []
total_interest = []
for i in np.arange(0,len(test)):
  tp = get_total_penalty(test.iloc[i]['BEGINNING BALANCE'], test.iloc[i]['FEES: AMOUNT.1'], 'false')
  ti = get_total_interest(test.iloc[i]['FEES: AMOUNT.1'], test.iloc[i]['FEES: INTEREST'], test.iloc[i]['FEES: EFFECTIVE DATE'], test.iloc[i]['FEES: END DATE'])
  total_penalty.append(tp)
  total_interest.append(ti)

test['total_pen'] = total_penalty
test['total_inter'] = total_interest

In [ ]:
test.to_excel("data.xlsx")

In [ ]:
ti = get_total_interest(test.iloc[0]['FEES: AMOUNT.1'], test.iloc[0]['FEES: INTEREST'], test.iloc[0]['FEES: EFFECTIVE DATE'], test.iloc[0]['FEES: END DATE'])
ti

**Upload data**

In [128]:
import mysql.connector
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

with open ('config.json') as f:
    configData = json.load(f)

engine = create_engine(configData['engine_url'])

mydb = mysql.connector.connect(
        host = configData['host'],
        user = configData['user'],
        password = configData['password'],
        database = configData['database']
    )

df = pd.read_excel('data.xlsx')

# Filter dataframe

df = df[df['STATUS'] == 'ACTIVE']
df = df[df['STATE'] == 'New Jersey']

print (len(df))

2069


In [165]:
column = {
    "MANAGING_COMPANY": "FUND",
    "CERTIFICATE": "CERTIFICATE",
    "INVESTMENT_DATE": "INVESTMENT DATE",
    "ORIGINAL_LIEN_AMOUNT": "BEGINNING BALANCE",
    "ORIGINAL_LIEN_INTEREST": "BEGINNING BALANCE DEDUCTED INTEREST",
    "ORIGINAL_LIEN_EFFECTIVE_DATE": "BEGINNING BALANCE EFFECTIVE DATE",
    "ORIGINAL_LIEN_INTERVAL": "FEES: INTERVAL", 
    "PREMIUM_AMOUNT": "PREMIUMS",
    "PREMIUM_INTEREST": "FEES: INTEREST", 
    "PREMIUM_EFFECTIVE_DATE": "FEES: EFFECTIVE DATE",
    "PREMIUM_INTERVAL": "FEES: INTERVAL",
    "TSRID": "TSRID",
    "STATE": "STATE",
    "COUNTY": "COUNTY",
    "MUNICIPALITY": "MUNICIPALITY",
    "PARCEL_ID": "PARCEL",
    "OWNER_NAME_CURRENT_OWNER": "PROPERTY OWNER",
    "OWNER_ADDRESS": "OWNER ADDRESS",
    "OWNER_CITY_STATE_ZIP": "OWNER CITY STATE ZIP",
    "HOMESTEAD_EXEMPTION": "HOMESTEAD EXEMPTION",
    "TOTAL_ASSESSED_VALUE": "ASSESSED VALUE",
    "TOTAL_MARKET_VALUE": "TOTAL MARKET VALUE",
    "APN": "APN",
    "LOCATION_FULL_STREET_ADDRESS": "OWNER ADDRESS",
    "LOCATION_CITY": "LOCATION CITY",
    "LOCATION_ZIP": "ZIP CODE",
    "LONGITUDE": "LONGITUDE",
    "LATITUDE": "LATITUDE",
    "OWNER_OCCUPIED": "OWNER OCCUPIED",
    "COUNTY_LAND_USE": "COUNTY LAND USE",
    "COUNTY_LAND_USE_DESC": "COUNTY LAND USE DESC",
    "STANDARDIZED_LAND_USE": "STANDARDIZED LAND USE",
    "LOT_SIZE": "LOT SIZE",
    "LOT_SIZE_UNIT": "LOT SIZE UNIT",
    "ZONING": "ZONING",
    "BUILDING_CLASS": "BUILDING CLASS",
    "YEAR_BUILT": "YEAR BUILT",
    "NO_OF_STORIES": "NO OF STORIES",
    "NO_OF_UNITS": "NO OF UNITS",
    "ASSESSMENT_YEAR": "ASSESSMENT YEAR",
    "LATEST_SALE_DATE": "LATEST SALE DATE",
    "LATEST_SALE_PRICE": "LATEST SALE PRICE",
    "LATEST_ARMS_LENGTH_SALE_DATE": "LATEST ARMS LENGTH SALE DATE",
    "LATEST_ARMS_LENGTH_SALE_PRICE": "LATEST ARMS LENGTH SALE PRICE",
    "PRIOR_ARMS_LENGTH_SALE_DATE": "PRIOR ARMS LENGTH SALE DATE",
    "PRIOR_ARMS_LENGTH_SALE_PRICE": "PRIOR ARMS LENGTH SALE PRICE",
    "LOAN1_AMOUNT": "LOAN1 AMOUNT",
    "LOAN1_DUE_DATE": "LOAN1 DUE DATE",
    "LOAN1_TYPE": "LOAN1 TYPE",
    "LOAN2_AMOUNT": "LOAN2 AMOUNT",
    "LEGAL_CITY": "LEGAL CITY",
    "LEGAL_BLOCK": "LEGAL BLOCK",
    "LEGAL_LOT_NUMBER": "LEGAL LOT NUMBER",
    "QUALIFIER": "QUALIFIER",
    "LEGAL_SECTION": "LEGAL SECTION",
    "LEGAL_UNIT": "LEGAL UNIT",
    "LEGAL_SUBDIVISION_NAME": "LEGAL SUBDIVISION NAME",
    "LEGAL_TRACT_NUMBER": "LEGAL TRACT NUMBER",
    "LEGAL_SECTION_TOWNSHIP_RANGE_MERIDIAN": "LEGAL SECTION TOWNSHIP RANGE MERIDIAN",
    "LEGAL_BRIEF_DESC": "LEGAL BRIEF DESC",
    "SUBJECT_FOUND_COUNT": "SUBJECT FOUND COUNT",
    "ENV_RISK": "ENV RISK",
    "TAG_KEEP_MAYBE_REMOVE_VIEWED": "TAG (KEEP, MAYBE, REMOVE, VIEWED)",
    "GRADE_A_B_C_D_F": "GRADE (A, B, C, D, F)",
    "GROUP_1_2_3_4_5": "GROUP (1, 2, 3, 4, 5)",
    "NOTES": "notes",
    "STATUS": "Status",
    "CREATE_DATE": "Created Date",
    "LAST_MODIFY_DATE": "Last Modified Date",
    "UNIQUE_ID": "REFERENCE ID",
    "CATEGORY": "FEES: CATEGORY",
    "AMOUNT": "FEES: AMOUNT",	
    "DESCRIPTION": "FEES: DESCRIPTION"
}
column_dict = dict((v,k) for k,v in column.items())
req_cols = list(column_dict.values())

df1 = df.copy()
df1 = df1[df1['FEES: CATEGORY'] == 'Beginning Balance']

df1 = df1.rename(columns=column_dict)
df1 = df1[df1.columns[df1.columns.isin(req_cols)]]
df1['ORIGINAL_LIEN_INTERVAL'] = df1['PREMIUM_INTERVAL']
df1['LOAN1_DUE_DATE'] = df1['LOAN1_DUE_DATE'].fillna("1994-01-01", inplace = True)

In [163]:
# df1 = df[df['PARCEL'] == '63 01017-0000-00002.14-C0028']

date_cols = ['ORIGINAL_LIEN_EFFECTIVE_DATE', 'PREMIUM_EFFECTIVE_DATE',  'LATEST_SALE_DATE', 'LOAN1_DUE_DATE']
for i in date_cols:
  df1[i] = pd.to_datetime(df1[i]).dt.strftime("%Y-%m-%d").astype(str)
df1['STATUS'] = 1
df1 = df1[df1['TSRID'].isnull() == False]
df1.columns


# df1.to_sql('PARCELS', engine, if_exists='append', index=False)

# df1.to_excel("test.xlsx")

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:1685: UserWarning: The provided table name 'PARCELS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [181]:
df2 = df.copy()
df2 = df2.rename(columns=column_dict)

df2 = df2[['UNIQUE_ID', 'CATEGORY', 'AMOUNT', 'DESCRIPTION', 'PREMIUM_EFFECTIVE_DATE', 'PREMIUM_INTERVAL', 'PREMIUM_INTEREST']]
df2 = df2[df2['CATEGORY'] == 'Subsequent Tax']
df2['CATEGORY'] = 3
df2['PREMIUM_EFFECTIVE_DATE'] = pd.to_datetime(df2['PREMIUM_EFFECTIVE_DATE']).dt.strftime("%Y-%m-%d").astype(str)
df2['DESCRIPTION'] = df2['DESCRIPTION'].astype(str)
df2['DESCRIPTION'] = df2['DESCRIPTION'].replace('nan','')
df2.columns = ['UNIQUE_ID', 'CATEGORY', 'AMOUNT', 'DESCRIPTION', 'EFFECTIVE_DATE', 'INTEREST_ACC_INTERVAL', 'INTEREST']



In [183]:
df2.to_sql('FEES', engine, if_exists='append', index=False)

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:1685: UserWarning: The provided table name 'FEES' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
